# LLM Tracing

With the Parea SDK, you can gain visibility into **any LLM application**. Together with the web application, Parea speeds up your debugging, evaluating, and monitoring workflows. Parea is also framework and provider agnostic. Parea traces your prompts and chains, whether deployed from Parea or within your codebase.

We will create a simple chat app and instrument logging with Parea. We will also add tags and other metadata to enrich our traces. The chat app uses three 'chained' components to generate a text argument on a provided subject:

1. An argument generation function
2. Critique function
3. Refine function

Each function will call an LLM provider; in our case, we'll use OpenAI, but you could quickly call Anthropic or Azure. Parea's dashboard shows how the LLM calls are organized for further analysis and investigation.

![DashboardDetailedView](img/dashboard_detailed_view.png)

Let's go!

## Prerequisites

First, install the Parea-ai SDK package. If you have an account with Parea, your LLM API Keys will be automatically used, so you won't need to redefine them here.
All you need is your Parea API key. Follow the instructions in the [docs](https://docs.parea.ai/api-reference/authentication) to get your api keys.

In [ ]:
# %pip install -U parea-ai > /dev/null

Next, configure the API Key in the environment to log traces to your account.

In [ ]:
import os

os.environ["PAREA_API_KEY"] = "<your-api-key>"

## Using the SDK

Next, define your chat application. The SDK will automatically generate an inference_id for each of your LLM calls. However, if you want to relate a chain of calls, you can use our helper function to create a trace_id. The function inputs, names, and other information are recorded and visible on Parea's dashboard. This is all done on a background thread to avoid blocking your app's execution.

We've created three prompts on Parea and have deployed them. Learn how to deploy a prompt [here](https://docs.parea.ai/deployments/deployments).

![Deployed_Prompts](img/deployed_prompts.png)

Now we only need the deployment id for each prompt to get started. You can also do this without a deployed prompt; we'll revisit this example in another walkthrough.

In [ ]:
from parea.utils.trace_utils import trace
from datetime import datetime

from parea import Parea
from parea.schemas.models import Completion

p = Parea(api_key=os.getenv("PAREA_API_KEY"))


# We pass the deployment_id and the required inputs to the completion function along with the trace_id
@trace
def argument_generator(query: str, additional_description: str = "") -> str:
    return p.completion(
        Completion(
            deployment_id="p-Ar-Oi14-nBxHUiradyql9",
            llm_inputs={
                "additional_description": additional_description,
                "date": f"{datetime.now()}",
                "query": query,
            },
        )
    ).content


@trace
def critic(argument: str) -> str:
    return p.completion(
        Completion(
            deployment_id="p-W2yPy93tAczYrxkipjli6",
            llm_inputs={"argument": argument},
        )
    ).content


@trace
def refiner(query: str, additional_description: str, current_arg: str, criticism: str) -> str:
    return p.completion(
        Completion(
            deployment_id="p-8Er1Xo0GDGF2xtpmMOpbn",
            llm_inputs={
                "additional_description": additional_description,
                "date": f"{datetime.now()}",
                "query": query,
                "current_arg": current_arg,
                "criticism": criticism,
            },
        )
    ).content


# This is the parent function which orchestrates the chaining. We'll define our trace_id and trace_name here
@trace
def argument_chain(query: str, additional_description: str = "") -> str:
    argument = argument_generator(query, additional_description)
    criticism = critic(argument)
    return refiner(query, additional_description, argument, criticism)

Now call the chain. If you set up your API key correctly at the start of this notebook, all the results should be traced to [Parea](https://www.optimusprompt.ai/dashboard). We will prompt the app to generate an argument that moonshine is good for you.

In [ ]:
result = argument_chain(
    "Whether moonshine is good for you.",
    additional_description="Provide a concise, few sentence argument on why moonshine is good for you.",
)
print(result)

## Recording feedback

The above is all you need to save your app's traces to Parea! You can try changing the functions or raising errors in the above code to see how it's visualized in [Parea](https://www.optimusprompt.ai/dashboard).

You can use the trace_id for other things like monitoring user feedback.

Below, our `argument_chain2` function is identical to the previous one except that we return the trace_id for use outside the function context.

In [ ]:
from parea.utils.trace_utils import get_current_trace_id


@trace
def argument_chain2(query: str, additional_description: str = "") -> tuple[str, str]:
    trace_id = get_current_trace_id()
    argument = argument_generator(query, additional_description)
    criticism = critic(argument)
    return refiner(query, additional_description, argument, criticism), trace_id

In [ ]:
result, trace_id = argument_chain2(
    "Whether moonshine is good for you.",
    additional_description="Provide a concise, few sentence argument on why moonshine is good for you.",
)
print(result)

With the trace_id, you can now log feedback from a user after the run is completed. Feedback score range from 0.0 (bad) to 1.0 (good).

In [ ]:
from parea.schemas.models import FeedbackRequest

p.record_feedback(FeedbackRequest(trace_id=trace_id, score=0.5))

![Feedback](./img/feedback.png)

## Enriching traces

One way to make your application traces more useful or actionable is to tag or add metadata to the logs. The completion function accepts additional properties such as:

- tags: List[str]
- metadata: Dict[str, str] - arbitrary key-value metadata
- target: str - a gold standard/expected output
- end_user_identifier: str - unique identifier for your end user

Below is an example.

In [16]:
from parea.schemas.models import CompletionResponse


# let's return the full CompletionResponse to see what other information is returned
@trace
def refiner2(query: str, additional_description: str, current_arg: str, criticism: str) -> CompletionResponse:
    return p.completion(
        Completion(
            deployment_id="p-8Er1Xo0GDGF2xtpmMOpbn",
            llm_inputs={
                "additional_description": additional_description,
                "date": f"{datetime.now()}",
                "query": query,
                "current_arg": current_arg,
                "criticism": criticism,
            },
        )
    )

In [17]:
# you can also add metadata and tags via the decorator
@trace(
    tags=["cookbook-example-deployed", "feedback_tracked-deployed"],
    metadata={"source": "python-sdk", "deployed": True},
)
def argument_chain3(query: str, additional_description: str = "") -> CompletionResponse:
    argument = argument_generator(query, additional_description)
    criticism = critic(argument)
    return refiner2(query, additional_description, argument, criticism)

In [ ]:
import json, attrs

result = argument_chain3(
    "Whether moonshine is good for you.",
    additional_description="Provide a concise, few sentence argument on why sunshine is good for you.",
)

p.record_feedback(
    FeedbackRequest(
        trace_id=result.trace_id,
        score=0.5,
        target="Moonshine is nice. Full stop.",
    )
)
print(json.dumps(attrs.asdict(result), indent=4))

Now you can navigate to the detailed logs with the trace_id to see the additional data.

![MetaData](./img/meta_data.png)

You can see your logs on the main dashboard and filter, search, and sort by various criteria.

![Dashboard](./img/dashboard.png)

## Recap
You made an example LLM application in this walkthrough and instrumented it using Parea's SDK.

You also added tags and metadata and even logged feedback to the logs. The SDK integrates wonderfully with your deployed prompts on Parea, keeping your code flexible and lightweight. Now you can iterate, debug, and monitor your application with ease.
